In [3]:
!pip install mlflow optuna statsmodels dvc -q
!pip install dvc[s3]
!apt install git -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154

In [4]:
import pandas as pd
import numpy as np
import mlflow
import os
import subprocess
import optuna
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

In [ ]:
MLFLOW_TRACKING_URI = ""
EXPERIMENT_NAME = "AQI hyperparameter & model testing (Manual)"
PREDICT_HORIZON = 72  # Predict next 3 days = 72 hours
VALIDATION_FRACTION = 0.2  # 20% holdout for final validation

In [6]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='/home/umair/mlruns/3', creation_time=1752022350617, experiment_id='3', last_update_time=1752022350617, lifecycle_stage='active', name='AQI hyperparameter & model testing (Manual)', tags={}>

In [7]:
!git init
!dvc init
!dvc remote add -d myremote s3://s3-bucket-umairrr

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                  

In [8]:
!git clone https://github.com/uma1r111/10pearls-AQI-Project-
%cd 10pearls-AQI-Project-

Cloning into '10pearls-AQI-Project-'...
remote: Enumerating objects: 416, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 416 (delta 87), reused 153 (delta 52), pack-reused 224 (from 1)
Receiving objects: 100% (416/416), 5.61 MiB | 11.55 MiB/s, done.
Resolving deltas: 100% (195/195), done.
/content/10pearls-AQI-Project-


In [ ]:
# Load and preprocess data
print("\nPulling latest feature_selection.csv from DVC remote (S3)...")
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_DEFAULT_REGION"] = ""
subprocess.run(["dvc", "pull"], check=True)


Pulling latest feature_selection.csv from DVC remote (S3)...


CompletedProcess(args=['dvc', 'pull'], returncode=0)

In [10]:
for root, dirs, files in os.walk(".", topdown=True):
    for name in files:
        print(os.path.join(root, name))

./karachi_weather_apr1_to_current.csv
./requirements.txt
./.dvcignore
./full_preprocessed_aqi_weather_data_with_all_features.csv
./feature_selection.csv
./README.md
./feature_selection.csv.dvc
./.gitignore
./fetch_daily_data.py
./Data Collection/karachi_weather_jun2025.ipynb
./Data Collection/karachi_weather_apr1_jun14.ipynb
./Data Collection/pollutants info Apr - Jun.ipynb
./Feature Selection/feature_selection.py
./Feature Selection/Feature_Selection.ipynb
./Feature Selection/initial_feature_extraction.ipynb
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.ipynb
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.txt
./Data/karachi_weather_apr1_jun14.csv
./Data/karachi_weather_jun2025.csv
./Data/pollutants info Mar - jul.csv
./Data Preprocessing/Data_PreProcessing.ipynb
./Data Preprocessing/run_preprocessing.py
./Data Preprocessing/data_quality_check.py
./.github/workflows/feature_engineering.yml
./.github/workflows/update_data.yml
./.github/

In [11]:
df = pd.read_csv("feature_selection.csv")

In [12]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df[(df["datetime"] >= "2025-04-01") & (df["datetime"] <= "2025-07-07")]
df = df.sort_values("datetime")

# Prepare full feature set (including AQI as part of exog for multivariate prediction)
features = df.drop(columns=["datetime"])
num_features = features.shape[1]

In [13]:
print(df.columns.tolist())

['datetime', 'aqi_us_lag1', 'aqi_us_lag12', 'aqi_us_lag24', 'pm2_5', 'log_pm10', 'scaled_humidity_%', 'scaled_temp_C_scaled_log_windspeed_kph', 'log_so2', 'day_of_week', 'scaled_temp_C', 'scaled_temp_C_scaled_o3', 'log_no2', 'aqi_us']


In [14]:
print(df["datetime"].head())
print("Filtered Rows:", len(df))

0   2025-04-01 00:00:00
1   2025-04-01 01:00:00
2   2025-04-01 02:00:00
3   2025-04-01 03:00:00
4   2025-04-01 04:00:00
Name: datetime, dtype: datetime64[ns]
Filtered Rows: 2329


# Sequencing for Walk Forward **Validation**

In [15]:
def walk_forward_validation(X, horizon, train_fraction=0.8):
    n_samples = len(X) - horizon + 1
    train_size = int(n_samples * train_fraction)
    train_X, val_X = [], []
    train_y, val_y = [], []

    for i in range(n_samples):
        seq_y = X[i:(i + horizon)].values
        seq_X = X[:i] if i > 0 else np.array([])  # Expanding window of past data
        if i < train_size:
            train_X.append(seq_X)
            train_y.append(seq_y)
        else:
            val_X.append(seq_X)
            val_y.append(seq_y)
    return (np.array(train_X, dtype=object), np.array(train_y),
            np.array(val_X, dtype=object), np.array(val_y))
X_train_seq, y_train, X_val_seq, y_val = walk_forward_validation(features, PREDICT_HORIZON)

In [16]:
# Flatten for SARIMAX input (SARIMAX expects 1D endog, 2D exog)
y_train_flat = y_train.reshape(y_train.shape[0], -1)
y_val_flat = y_val.reshape(y_val.shape[0], -1)

In [17]:
train_exog = features.iloc[:len(y_train)].values
val_exog = features.iloc[len(y_train):len(y_train) + len(y_val)].values
print("y_train_flat shape:", y_train_flat.shape)
print("y_val_flat shape:", y_val_flat.shape)

y_train_flat shape: (1806, 936)
y_val_flat shape: (452, 936)


In [18]:
# Explicitly separate target ("AQI") and exogenous features
target_col = "aqi_us"
target = features[target_col]
exog_features = features.drop(columns=[target_col])

In [19]:
# Split full data for final validation
train_size_full = int(len(features) * (1 - VALIDATION_FRACTION))
train_target = target.iloc[:train_size_full]
val_target = target.iloc[train_size_full:]
train_exog_full = exog_features.iloc[:train_size_full].values
val_exog_full = exog_features.iloc[train_size_full:].values

# Optuna Hyperparameter **Tuning**

In [ ]:
# Optuna objective function with walk-forward validation
def optuna_objective(trial):
    p = trial.suggest_int("p", 0, 2)
    d = trial.suggest_int("d", 0, 2)
    q = trial.suggest_int("q", 0, 2)
    P = trial.suggest_int("P", 0, 2)
    D = trial.suggest_int("D", 0, 1)
    Q = trial.suggest_int("Q", 0, 2)
    seasonal_period = trial.suggest_categorical("seasonal_period", [24, 48, 72])

    model = SARIMAX(endog=train_target,
                    exog=train_exog_full,
                    order=(p, d, q),
                    seasonal_order=(P, D, Q, seasonal_period),
                    enforce_stationarity=False,
                    enforce_invertibility=False)
    try:
        fitted_model = model.fit(disp=0)
        val_preds = fitted_model.forecast(steps=len(val_target), exog=val_exog_full)
        rmse = sqrt(mean_squared_error(val_target, val_preds))
        return rmse
    except:
        return float("inf")


# Run optimization
study = optuna.create_study(direction="minimize")
study.optimize(optuna_objective, n_trials=15)

# Log best trial to MLflow
with mlflow.start_run(run_name="SARIMAX_Best_Hyperparameters_Optuna"):
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)
    print("Best parameters:", study.best_params)

[I 2025-07-11 00:00:27,486] A new study created in memory with name: no-name-50629949-660d-41a7-ac71-86a5311c36ab
[I 2025-07-11 00:27:41,940] Trial 0 finished with value: 1.263218010651326 and parameters: {'p': 1, 'd': 0, 'q': 0, 'P': 1, 'D': 1, 'Q': 2, 'seasonal_period': 48}. Best is trial 0 with value: 1.263218010651326.
[I 2025-07-11 00:40:32,825] Trial 1 finished with value: 28.34467027335216 and parameters: {'p': 2, 'd': 2, 'q': 1, 'P': 2, 'D': 0, 'Q': 0, 'seasonal_period': 48}. Best is trial 0 with value: 1.263218010651326.
[I 2025-07-11 00:41:17,604] Trial 2 finished with value: 10.645931545308235 and parameters: {'p': 2, 'd': 1, 'q': 2, 'P': 0, 'D': 0, 'Q': 1, 'seasonal_period': 24}. Best is trial 0 with value: 1.263218010651326.
[I 2025-07-11 00:45:01,207] Trial 3 finished with value: 2.6399395824996605 and parameters: {'p': 1, 'd': 0, 'q': 1, 'P': 0, 'D': 1, 'Q': 0, 'seasonal_period': 72}. Best is trial 0 with value: 1.263218010651326.
[I 2025-07-11 01:15:07,222] Trial 4 fini

In [20]:
best_params = {
    "p": 2,
    "d": 0,
    "q": 1,
    "P": 1,
    "D": 0,
    "Q": 2,
    "seasonal_period": 48
}

# SARIMAX Model **Fit**

In [22]:
#best_params = study.best_params
with mlflow.start_run(run_name="Final_SARIMAX_Model_Optuna"):
    # Fit final model with best parameters on training data (excluding validation holdout)
    final_model = SARIMAX(endog=train_target,  # 'aqi_us' from index 12
                          exog=train_exog_full[:, :12],  # First 12 columns as exog
                          order=(best_params["p"], best_params["d"], best_params["q"]),
                          seasonal_order=(best_params["P"], best_params["D"], best_params["Q"], best_params["seasonal_period"]),
                          enforce_stationarity=False,
                          enforce_invertibility=False)
    fitted_model = final_model.fit(disp=0)

    # Predict on validation holdout for final metrics
    val_preds = fitted_model.forecast(steps=len(val_target), exog=val_exog_full[:, :12])
    val_preds_full = np.column_stack((val_preds, val_exog_full[:, :12]))  # Reconstruct all features
    rmse = sqrt(mean_squared_error(val_target, val_preds))
    mae = mean_absolute_error(val_target, val_preds)

    # Log to MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("final_rmse", rmse)
    mlflow.log_metric("final_mae", mae)
    mlflow.set_tag("model_type", "SARIMAX_Multivariate")
    mlflow.set_tag("tuner", "Optuna")
    print("Final RMSE:", rmse)
    print("Final MAE:", mae)

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Final RMSE: 1.1390574026250793
Final MAE: 0.7723148161877342
🏃 View run Final_SARIMAX_Model_Optuna at: http://172.174.154.85:8000/#/experiments/3/runs/5c7f9ca8a663485b86fa654b8c1d936c
🧪 View experiment at: http://172.174.154.85:8000/#/experiments/3


# **Prediction**

In [23]:
# Prepare last sequence for forecasting
last_exog = exog_features.iloc[-PREDICT_HORIZON:].values  # Last 72 hours of exogenous data

# Forecast next 72 hours for AQI
future_preds = fitted_model.forecast(steps=PREDICT_HORIZON, exog=last_exog)
future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),
                             periods=PREDICT_HORIZON, freq='H')

future_df = pd.DataFrame({target_col: future_preds, "datetime": future_dates})
future_df.to_csv("future_AQI_predictions_sarimax_optuna.csv", index=False)
mlflow.log_artifact("future_AQI_predictions_sarimax_optuna.csv")


/tmp/ipython-input-23-597289894.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),
